In [33]:
#!/usr/bin/env python
# coding: utf-8

import math
import random
import os
import json
from importlib import reload


import inspect
import code

import torch
from torch.nn.functional import cross_entropy, log_softmax

import datasets
from util import const, icl, lm, infogain, util

In [38]:
reload(infogain)

<module 'util.infogain' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/infogain.py'>

In [92]:
args = type('', (), {})()
# parser.add_argument('out_file', type=str)
# args.out_file = NotImplemented
# parser.add_argument('--n_tries', type=int, default=10)
args.n_tries = NotImplemented
# parser.add_argument('--p', type=float, default=0.5)
args.p = NotImplemented
# parser.add_argument('--gen_len', type=int, default=50)
args.gen_len = NotImplemented
# parser.add_argument('--model', type=str, default='n125m')
args.model = '2l'
# parser.add_argument('--n_demos', type=int, default=10)
args.n_demos = NotImplemented
# parser.add_argument('--n_gen_bs', type=int, default=10)
args.n_gen_bs = NotImplemented
# parser.add_argument('--device', type=str, default='cuda')
args.device = 'cuda'
# parser.add_argument('--seed', type=int, default=0)
args.seed = 0
# parser.add_argument('--worker_id', type=int, default=None)
args.worker_id = NotImplemented
# parser.add_argument('--n_shards', type=int, default=None)
args.n_shards = NotImplemented
# parser.add_argument('--print_int', type=int, default=1)
args.print_int = NotImplemented
# parser.add_argument('--override', action='store_true')
args.override = NotImplemented

# if args.override:
#     with open(args.out_file, 'w') as out:
#         pass
# else:
#     assert(not os.path.exists(args.out_file))

util.set_all_seeds(args.seed)

In [93]:
model, tokenizer, cw_length = lm.load(args.model, args.device)

In [4]:
with open('data/multirc/dev.json') as f:
  data = json.load(f)['data']

In [73]:
reload(infogain)

<module 'util.infogain' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/infogain.py'>

In [94]:
with torch.no_grad():
  all_lls = []
  pres, recs = [], []
  for datum_idx, datum in enumerate(data):
    print(datum_idx)
    datum = datum['paragraph']
    passage_text = datum['text']
    passage_ids = tokenizer('\n' + passage_text.strip()).input_ids
    datum_lls = []
    for question in datum['questions']:
      true_positives, pred_positives, correct = 0, 0, 0
      question_text = question['question']
      answers = question['answers']
      tests = multirc_make_tests(question_text, [a['text'] for a in answers])
      question_lls = []
      for answer, test_tuple in zip(answers, tests):
        tests = [test_tuple[0]] + test_tuple[1]
        test_ids = [ tokenizer(test).input_ids for test in tests ]
        pad_lens = infogain.get_pad_lens(test_ids)
        for test_idx in range(len(test_ids)):
          test_ids[test_idx] += passage_ids + [0]*pad_lens[test_idx]
        tests_t = torch.LongTensor(test_ids).to(args.device)
        result = model(tests_t)
        token_lls = cross_entropy(result.logits[:, :-1].transpose(1,2), tests_t[:, 1:], reduction='none').mul(-1)
        lls = [ l[-len(passage_ids)-pad_lens[i]:-pad_lens[i]].sum().item() for i, l in enumerate(token_lls) ]
        if lls[0]-lls[1] > 0 and lls[0]-lls[2] > 0:
          pred = True
        else:
          pred = False
        pred_positives += pred
        true_positives += answer['isAnswer']
        correct += pred and answer['isAnswer']
        question_lls.append(lls)
      if true_positives > 0:
        pre = correct / pred_positives if pred_positives > 0 else 0
        rec = correct / true_positives
      else:
        pre, rec = 1, 1
      pres.append(pre)
      recs.append(rec)
      datum_lls.append(question_lls)
    all_lls.append(datum_lls)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82


In [29]:
def avg(l):
  return sum(l) / len(l)

In [101]:
2 * (avg(pres) * avg(recs)) / (avg(pres) + avg(recs))

0.4884700444330475

In [70]:
torch.randn(10)[-9:0]

tensor([])

In [40]:
dir(infogain)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'e2e_gen_tests',
 'get_pad_lens',
 'multirc_make_tests',
 'multirc_proc_passage',
 'random',
 're']

In [139]:
for threshold in range(0, 400, 10):
  with torch.no_grad():
    pres, recs = [], []
    acc = []
    for datum_idx, datum in enumerate(data):
      datum = datum['paragraph']
      passage_text = datum['text']
      passage_ids = tokenizer('\n' + passage_text.strip()).input_ids
      datum_lls = all_lls[datum_idx]
      for question_idx, question in enumerate(datum['questions']):
        true_positives, pred_positives, correct = 0, 0, 0
        question_text = question['question']
        answers = question['answers']
        tests = multirc_make_tests(question_text, [a['text'] for a in answers])
        question_lls = datum_lls[question_idx]
        all_right = True
        for answer_idx, (answer, test_tuple) in enumerate(zip(answers, tests)):
          answer_lls = question_lls[answer_idx]
          pred =  (answer_lls[0] - answer_lls[1] < threshold) and (answer_lls[0] - answer_lls[2] < threshold)
          #pred = (answer_lls[0] - answer_lls[2] > threshold)
          pred_positives += pred
          true_positives += answer['isAnswer']
          correct += pred and answer['isAnswer']
          all_right = all_right and (pred == answer['isAnswer'])
        acc.append(all_right)
        if true_positives > 0:
          pre = correct / pred_positives if pred_positives > 0 else 0
          rec = correct / true_positives
        else:
          pre, rec = 1, 1
        pres.append(pre)
        recs.append(rec)
  print(2 * (avg(pres) * avg(recs)) / (avg(pres) + avg(recs)))
  # print(avg(acc))

0.013432179816765654
0.1568466289168926
0.3941507296769915
0.5041326290812322
0.5586778170777598
0.5851905495119936
0.598158998274806
0.6011063777309774
0.606431049473297
0.608496633088301
0.6102838757657123
0.6103828897550012
0.6107981409872096
0.6121160912774928
0.6122779667982298
0.6122779667982298
0.6123973692330503
0.6123973692330503
0.6123973692330503
0.6127964690511252
0.6127964690511252
0.6127964690511252
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885
0.6130464512432885


In [91]:
n125m_lls = all_lls

In [120]:
import numpy as np

In [121]:
np.linspace(0,1 )

array([0.        , 0.02040816, 0.04081633, 0.06122449, 0.08163265,
       0.10204082, 0.12244898, 0.14285714, 0.16326531, 0.18367347,
       0.20408163, 0.2244898 , 0.24489796, 0.26530612, 0.28571429,
       0.30612245, 0.32653061, 0.34693878, 0.36734694, 0.3877551 ,
       0.40816327, 0.42857143, 0.44897959, 0.46938776, 0.48979592,
       0.51020408, 0.53061224, 0.55102041, 0.57142857, 0.59183673,
       0.6122449 , 0.63265306, 0.65306122, 0.67346939, 0.69387755,
       0.71428571, 0.73469388, 0.75510204, 0.7755102 , 0.79591837,
       0.81632653, 0.83673469, 0.85714286, 0.87755102, 0.89795918,
       0.91836735, 0.93877551, 0.95918367, 0.97959184, 1.        ])

In [148]:
#f1a
for threshold in np.linspace(-7, -4):
  with torch.no_grad():
    preds, corrects, grounds = [], [], []
    for datum_idx, datum in enumerate(data):
      datum = datum['paragraph']
      passage_text = datum['text']
      passage_ids = tokenizer('\n' + passage_text.strip()).input_ids
      datum_lls = all_lls[datum_idx]
      for question_idx, question in enumerate(datum['questions']):
        true_positives, pred_positives, correct = 0, 0, 0
        question_text = question['question']
        answers = question['answers']
        tests = multirc_make_tests(question_text, [a['text'] for a in answers])
        question_lls = datum_lls[question_idx]
        all_right = True
        for answer_idx, (answer, test_tuple) in enumerate(zip(answers, tests)):
          answer_lls = question_lls[answer_idx]
          pred =  (answer_lls[0] - answer_lls[1] > threshold) and (answer_lls[0] - answer_lls[2] > threshold)
          #pred = (answer_lls[0] - answer_lls[2] > threshold)
          preds.append(pred)
          corrects.append(pred == answer['isAnswer'] and answer['isAnswer'])
          grounds.append(answer['isAnswer'])
  pre = sum(corrects) / sum(preds)
  rec = sum(corrects) / sum(grounds)
  print(2 * (pre * rec) / (pre + rec))
  # print(avg(acc))

0.6211819389110226
0.621630615640599
0.6219552886219554
0.6221776216758655
0.6225432555014279
0.6230834035383319
0.6236704372784061
0.6242362525458248
0.6237876467585503
0.6238688748506062
0.6245926942205454
0.6259463179628355
0.6251079260922121
0.6251082251082251
0.6255215577190543
0.6256535378180551
0.6256338520720406
0.6262272089761571
0.6269548409769813
0.6278659611992946
0.6278740714538379
0.6283484122760333
0.6278282558346695
0.6270096463022509
0.62705798138869
0.6250898634076204
0.624977473418634
0.623710407239819
0.624136677571792
0.6236323851203501
0.6233528550512445
0.6225895316804408
0.6206642066420663
0.6213160333642261
0.6197654941373534
0.6194029850746269
0.6194756554307115
0.6196653506298176
0.6183465458663647
0.6179817905918057
0.6167619047619047
0.6150315547905909
0.6145873320537427
0.6155029813425659
0.6160077145612344
0.6165471807789187
0.6154743390357699
0.6155945419103315
0.6141793967880925
0.6129095546399843
